In [1]:
import pandas as pd
path="../dataset"
data = pd.read_csv(path + '/CAvideos.csv')
data.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787425,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
1,0dBIkQ4Mz1M,17.14.11,PLUSH - Bad Unboxing Fan Mail,iDubbbzTV,23,2017-11-13T17:00:00.000Z,"plush|""bad unboxing""|""unboxing""|""fan mail""|""id...",1014651,127794,1688,13030,https://i.ytimg.com/vi/0dBIkQ4Mz1M/default.jpg,False,False,False,STill got a lot of packages. Probably will las...
2,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Le...",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,"racist superman|""rudy""|""mancuso""|""king""|""bach""...",3191434,146035,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► http...
3,d380meD0W0M,17.14.11,I Dare You: GOING BALD!?,nigahiga,24,2017-11-12T18:01:41.000Z,"ryan|""higa""|""higatv""|""nigahiga""|""i dare you""|""...",2095828,132239,1989,17518,https://i.ytimg.com/vi/d380meD0W0M/default.jpg,False,False,False,I know it's been a while since we did this sho...
4,2Vv-BfVoq4g,17.14.11,Ed Sheeran - Perfect (Official Music Video),Ed Sheeran,10,2017-11-09T11:04:14.000Z,"edsheeran|""ed sheeran""|""acoustic""|""live""|""cove...",33523622,1634130,21082,85067,https://i.ytimg.com/vi/2Vv-BfVoq4g/default.jpg,False,False,False,🎧: https://ad.gt/yt-perfect\n💰: https://atlant...


In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

data['text'] = data['title'] + ' ' + data['tags']
vectorizer = CountVectorizer(stop_words='english', max_features=100)
X = vectorizer.fit_transform(data['text'].fillna(''))


In [3]:
word_freq = np.asarray(X.sum(axis=0)).flatten()
words = vectorizer.get_feature_names_out()
freq_df = pd.DataFrame({'word': words, 'frequency': word_freq}).sort_values(by='frequency', ascending=False)

In [4]:
freq_df.head(10)

,word,frequency
4,2018,15955
59,news,10220
58,new,9662
91,video,8228
27,funny,8204
88,trump,7279
94,vs,7214
76,songs,7042
3,2017,6812
20,episode,6808


In [5]:
X_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
y = data['views']

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [7]:
feature_importances = pd.DataFrame({
    'word': X_df.columns,
    'importance': model.feature_importances_
}).sort_values(by='importance', ascending=False)

print(feature_importances.head(20))

             word  importance
61       official    0.090817
70          rhett    0.064870
95            war    0.051872
91          video    0.051649
58            new    0.039501
47           link    0.038401
79        stephen    0.037600
81          super    0.033172
87        trailer    0.031000
56          music    0.029190
3            2017    0.028721
75           song    0.025254
98        youtube    0.024631
50           love    0.024302
97          world    0.023618
4            2018    0.020401
54          movie    0.018075
19  entertainment    0.017308
27          funny    0.017181
49          logan    0.015951


In [8]:
y = data['category_id']

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

data['text'] = data['title'] + ' ' + data['tags']
data = data.dropna(subset=['text', 'category_id'])

X = data['text']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [10]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000, random_state=42)

model.fit(X_train_tfidf, y_train)

from sklearn.metrics import classification_report, accuracy_score
y_pred = model.predict(X_test_tfidf)
print("Classification Report:")
print(classification_report(y_test, y_pred))

print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")


Classification Report:
              precision    recall  f1-score   support

           1       0.77      0.54      0.64       406
           2       0.85      0.61      0.71        67
          10       0.84      0.84      0.84       765
          15       0.85      0.77      0.81        87
          17       0.90      0.91      0.90       539
          19       0.91      0.80      0.85        79
          20       0.87      0.60      0.71       294
          22       0.59      0.43      0.50       840
          23       0.76      0.72      0.74       726
          24       0.69      0.86      0.76      2682
          25       0.78      0.77      0.77       818
          26       0.82      0.78      0.80       417
          27       0.84      0.49      0.62       211
          28       0.73      0.69      0.71       209
          29       1.00      0.33      0.50        15
          30       0.00      0.00      0.00         2
          43       0.56      0.50      0.53        20

   

/usr/local/anaconda3/envs/cse258/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/anaconda3/envs/cse258/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/anaconda3/envs/cse258/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

In [11]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_tfidf, y_train)

from sklearn.metrics import classification_report, accuracy_score
y_pred = model.predict(X_test_tfidf)
print("Classification Report:")
print(classification_report(y_test, y_pred))

print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")


Classification Report:
              precision    recall  f1-score   support

           1       0.95      0.78      0.86       406
           2       0.96      0.81      0.88        67
          10       0.94      0.96      0.95       765
          15       0.97      0.84      0.90        87
          17       0.93      0.96      0.94       539
          19       0.89      0.92      0.91        79
          20       0.95      0.84      0.89       294
          22       0.71      0.80      0.75       840
          23       0.91      0.89      0.90       726
          24       0.90      0.92      0.91      2682
          25       0.87      0.87      0.87       818
          26       0.94      0.92      0.93       417
          27       0.92      0.81      0.86       211
          28       0.88      0.84      0.86       209
          29       1.00      0.73      0.85        15
          30       1.00      1.00      1.00         2
          43       0.65      0.65      0.65        20

   

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "Support Vector Machine": SVC(kernel='linear', random_state=42),
    "Naive Bayes": MultinomialNB(),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

for name, clf in models.items():
    clf.fit(X_train_tfidf, y_train)
    y_pred = clf.predict(X_test_tfidf)
    print(f"{name} Accuracy: {accuracy_score(y_test, y_pred):.2f}")

Logistic Regression Accuracy: 0.75
Support Vector Machine Accuracy: 0.76
Naive Bayes Accuracy: 0.69
Random Forest Accuracy: 0.89
